<a href="https://colab.research.google.com/github/akf2691/Cryptography-Using_Diffie-Hellman_AES_RSA/blob/main/Cryptography_Assessment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q cryptography


In [2]:
%%bash
cat > dh_aes_rsa.py <<'PY'
import os
from typing import Tuple

from cryptography.hazmat.primitives.asymmetric.x25519 import (
    X25519PrivateKey, X25519PublicKey
)
from cryptography.hazmat.primitives.kdf.hkdf import HKDF
from cryptography.hazmat.primitives import hashes, serialization
from cryptography.hazmat.primitives.ciphers.aead import AESGCM

from cryptography.hazmat.primitives.asymmetric import rsa, padding
from cryptography.hazmat.primitives import hashes as hmac_hashes

# Parameters
HKDF_SALT_SIZE = 16
NONCE_SIZE = 12
AES_KEY_LEN = 32  # 256-bit AES

# X25519 is a high-speed, high-security protocol used for Diffie-Hellman key exchange.
def generate_x25519_keypair() -> Tuple[X25519PrivateKey, X25519PublicKey]:
    sk = X25519PrivateKey.generate()
    pk = sk.public_key()
    return sk, pk

def save_x25519_private_raw(sk: X25519PrivateKey, path: str) -> None:
    data = sk.private_bytes(
        encoding=serialization.Encoding.Raw,
        format=serialization.PrivateFormat.Raw,
        encryption_algorithm=serialization.NoEncryption()
    )
    with open(path, "wb") as f:
        f.write(data)

def load_x25519_private_raw(path: str) -> X25519PrivateKey:
    raw = open(path, "rb").read()
    return X25519PrivateKey.from_private_bytes(raw)

def save_x25519_public_raw(pk: X25519PublicKey, path: str) -> None:
    data = pk.public_bytes(
        encoding=serialization.Encoding.Raw,
        format=serialization.PublicFormat.Raw
    )
    with open(path, "wb") as f:
        f.write(data)

def load_x25519_public_raw(path: str) -> X25519PublicKey:
    raw = open(path, "rb").read()
    return X25519PublicKey.from_public_bytes(raw)

# HKDF (derive AES key from shared secret)
def derive_aes_key(shared_secret: bytes, salt: bytes) -> bytes:
    hkdf = HKDF(
        algorithm=hashes.SHA256(),
        length=AES_KEY_LEN,
        salt=salt,
        info=b"dh-aes-gcm",
    )
    return hkdf.derive(shared_secret)

# RSA helpers (sign/verify)
def generate_rsa_keypair(key_size: int = 2048):
    private_key = rsa.generate_private_key(public_exponent=65537, key_size=key_size)
    public_key = private_key.public_key()
    return private_key, public_key

def save_rsa_private(priv, path: str):
    with open(path, "wb") as f:
        f.write(priv.private_bytes(
            encoding=serialization.Encoding.PEM,
            format=serialization.PrivateFormat.PKCS8,
            encryption_algorithm=serialization.NoEncryption()
        ))

def save_rsa_public(pub, path: str):
    with open(path, "wb") as f:
        f.write(pub.public_bytes(
            encoding=serialization.Encoding.PEM,
            format=serialization.PublicFormat.SubjectPublicKeyInfo
        ))

def load_rsa_public(path: str):
    data = open(path, "rb").read()
    return serialization.load_pem_public_key(data)

def load_rsa_private(path: str):
    data = open(path, "rb").read()
    return serialization.load_pem_private_key(data, password=None)

def sign_data(priv_key, data: bytes) -> bytes:
    sig = priv_key.sign(
        data,
        padding.PSS(mgf=padding.MGF1(hmac_hashes.SHA256()), salt_length=padding.PSS.MAX_LENGTH),
        hmac_hashes.SHA256()
    )
    return sig

def verify_signature(pub_key, data: bytes, signature: bytes) -> bool:
    try:
        pub_key.verify(
            signature,
            data,
            padding.PSS(mgf=padding.MGF1(hmac_hashes.SHA256()), salt_length=padding.PSS.MAX_LENGTH),
            hmac_hashes.SHA256()
        )
        return True
    except Exception:
        return False

# AES encryption using key derived from DH
def encrypt_file_with_dh(sender_x25519_sk: X25519PrivateKey,
                         receiver_x25519_pub: X25519PublicKey,
                         sender_rsa_priv,
                         in_path: str,
                         out_enc_path: str,
                         out_sig_path: str,
                         out_sender_pub_path: str) -> None:
    # Read plaintext
    plaintext = open(in_path, "rb").read()

    # Derive shared secret
    shared_secret = sender_x25519_sk.exchange(receiver_x25519_pub)

    # Salt for HKDF
    salt = os.urandom(HKDF_SALT_SIZE)
    key = derive_aes_key(shared_secret, salt)

    # AES-GCM encrypt
    nonce = os.urandom(NONCE_SIZE)
    aesgcm = AESGCM(key)
    ciphertext = aesgcm.encrypt(nonce, plaintext, associated_data=None)  # ciphertext||tag

    # Write encrypted file (salt | nonce | ciphertext)
    with open(out_enc_path, "wb") as f:
        f.write(salt + nonce + ciphertext)

    # Sign the encrypted file bytes using RSA private key
    sig = sign_data(sender_rsa_priv, salt + nonce + ciphertext)
    with open(out_sig_path, "wb") as f:
        f.write(sig)

    # Save sender's X25519 public (raw bytes) for receiver to compute shared secret
    save_x25519_public_raw(sender_x25519_sk.public_key(), out_sender_pub_path)

def decrypt_file_with_dh(receiver_x25519_sk: X25519PrivateKey,
                         sender_x25519_pub_path: str,
                         sender_rsa_pub_path: str,
                         enc_path: str,
                         sig_path: str,
                         out_plain_path: str) -> None:
    # Load files
    enc_data = open(enc_path, "rb").read()
    sig = open(sig_path, "rb").read()
    sender_pub = load_x25519_public_raw(sender_x25519_pub_path)
    sender_rsa_pub = load_rsa_public(sender_rsa_pub_path)

    # Verify signature
    if not verify_signature(sender_rsa_pub, enc_data, sig):
        raise ValueError("Signature verification failed. Aborting decryption.")

    # Parse salt, ciphertext
    salt = enc_data[:HKDF_SALT_SIZE]
    nonce = enc_data[HKDF_SALT_SIZE:HKDF_SALT_SIZE + NONCE_SIZE]
    ciphertext = enc_data[HKDF_SALT_SIZE + NONCE_SIZE:]

    # Derive shared secret
    shared_secret = receiver_x25519_sk.exchange(sender_pub)
    key = derive_aes_key(shared_secret, salt)

    # Decrypt
    aesgcm = AESGCM(key)
    plaintext = aesgcm.decrypt(nonce, ciphertext, associated_data=None)

    with open(out_plain_path, "wb") as f:
        f.write(plaintext)

PY


In [3]:
from dh_aes_rsa import generate_x25519_keypair, save_x25519_private_raw, save_x25519_public_raw
import os

os.makedirs("keys", exist_ok=True)

# Generate receiver keypair and save raw bytes
receiver_sk, receiver_pk = generate_x25519_keypair()
save_x25519_private_raw(receiver_sk, "keys/receiver_x25519_private.raw")
save_x25519_public_raw(receiver_pk, "keys/receiver_x25519_public.raw")

print("Receiver X25519 keypair generated and saved")


Receiver X25519 keypair generated and saved


In [4]:
from dh_aes_rsa import generate_rsa_keypair, save_rsa_private, save_rsa_public
import os

os.makedirs("keys", exist_ok=True)

sender_rsa_priv, sender_rsa_pub = generate_rsa_keypair()
save_rsa_private(sender_rsa_priv, "keys/sender_rsa_private.pem")
save_rsa_public(sender_rsa_pub, "keys/sender_rsa_public.pem")

print("Sender RSA keypair generated and saved")


Sender RSA keypair generated and saved


In [6]:
from dh_aes_rsa import generate_x25519_keypair, load_x25519_public_raw, save_x25519_public_raw, load_rsa_private
from dh_aes_rsa import encrypt_file_with_dh
import os

# Generate ephemeral sender X25519 keypair (ephemeral per message)
sender_sk, sender_pk = generate_x25519_keypair()

# Save sender public (so receiver can use it)
os.makedirs("encrypted", exist_ok=True)
save_x25519_public_raw(sender_pk, "encrypted/sender_x25519_pub.raw")

# Load receiver public (saved earlier)
receiver_pub = load_x25519_public_raw("keys/receiver_x25519_public.raw")

# Load sender RSA private (to sign)
sender_rsa_priv = load_rsa_private("keys/sender_rsa_private.pem")

# File to encrypt
in_path = "/content/message"
out_enc = f"encrypted/{os.path.basename(in_path)}.enc"
out_sig = f"encrypted/{os.path.basename(in_path)}.sig"
out_sender_pub = "encrypted/sender_x25519_pub.raw"

encrypt_file_with_dh(sender_sk, receiver_pub, sender_rsa_priv, in_path, out_enc, out_sig, out_sender_pub)
# Save sender RSA public for receiver to verify
from dh_aes_rsa import save_rsa_public
save_rsa_public(sender_rsa_pub, "encrypted/sender_rsa_public.pem")

print("Encrypted and signed.")
print(" -", out_enc)
print(" -", out_sig)
print(" - encrypted/sender_x25519_pub.raw")
print(" - encrypted/sender_rsa_public.pem")


Encrypted and signed.
 - encrypted/message.enc
 - encrypted/message.sig
 - encrypted/sender_x25519_pub.raw
 - encrypted/sender_rsa_public.pem


In [7]:
 from dh_aes_rsa import decrypt_file_with_dh, load_x25519_private_raw
import os

# Ensure decrypted output dir
os.makedirs("decrypted", exist_ok=True)


enc_path = input("Enter encrypted file path (e.g., encrypted/message.txt.enc): ").strip()
sig_path = input("Enter signature file path (e.g., encrypted/message.txt.sig): ").strip()
sender_x25519_pub_path = input("Enter sender's x25519 pub path (e.g., encrypted/sender_x25519_pub.raw): ").strip()
sender_rsa_pub_path = input("Enter sender RSA public path (e.g., encrypted/sender_rsa_public.pem): ").strip()
out_plain = input("Enter output path for decrypted file (e.g., decrypted/message_decrypted.txt): ").strip()



try:
    decrypt_file_with_dh(receiver_sk, sender_x25519_pub_path, sender_rsa_pub_path, enc_path, sig_path, out_plain)
    print("Decryption successful. Plaintext saved to:", out_plain)
except Exception as e:
    print("Decryption failed:", str(e))


Enter encrypted file path (e.g., encrypted/message.txt.enc): /content/encrypted/message.enc
Enter signature file path (e.g., encrypted/message.txt.sig): /content/encrypted/message.sig
Enter sender's x25519 pub path (e.g., encrypted/sender_x25519_pub.raw): /content/encrypted/sender_x25519_pub.raw
Enter sender RSA public path (e.g., encrypted/sender_rsa_public.pem): /content/encrypted/sender_rsa_public.pem
Enter output path for decrypted file (e.g., decrypted/message_decrypted.txt): /content/decrypted/msg
Decryption successful. Plaintext saved to: /content/decrypted/msg
